# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

In this project i used the events (goals and results) of the current bundesliga season. This dataset is chosen, becouse to create a chat bot that can give accurate facts of the current season.

Be carefully. There is not limit of context. Every data gets send to the module. It could be expensive...

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [5]:
# install required modules
%pip install requests openai scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# imports
import os
import requests
import json
import pandas as pd


In [ ]:
# set internet proxy
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""
os.environ["NO_PROXY"] = "localhost"

In [6]:
# pull data
match_days = 34
data_dir = "./data"
year = 2024
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

for i in range(1, match_days + 1):
    url = f"https://api.openligadb.de/getmatchdata/bl1/{year}/{i}"
    file_name = f"match_day_{i}.json"
    file_path = os.path.join(data_dir, file_name)
    
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(response.json(), f, ensure_ascii=False, indent=4)
        print(f"Matchday {i} successfully saved: {file_path}")
    else:
        print(f"Error {i}: HTTP {response.status_code}")

Matchday 1 successfully saved: ./data\match_day_1.json
Matchday 2 successfully saved: ./data\match_day_2.json
Matchday 3 successfully saved: ./data\match_day_3.json
Matchday 4 successfully saved: ./data\match_day_4.json
Matchday 5 successfully saved: ./data\match_day_5.json
Matchday 6 successfully saved: ./data\match_day_6.json
Matchday 7 successfully saved: ./data\match_day_7.json
Matchday 8 successfully saved: ./data\match_day_8.json
Matchday 9 successfully saved: ./data\match_day_9.json
Matchday 10 successfully saved: ./data\match_day_10.json
Matchday 11 successfully saved: ./data\match_day_11.json
Matchday 12 successfully saved: ./data\match_day_12.json
Matchday 13 successfully saved: ./data\match_day_13.json
Matchday 14 successfully saved: ./data\match_day_14.json
Matchday 15 successfully saved: ./data\match_day_15.json
Matchday 16 successfully saved: ./data\match_day_16.json
Matchday 17 successfully saved: ./data\match_day_17.json
Matchday 18 successfully saved: ./data\match_day_

In [ ]:
texts = []

for i in range(1, match_days + 1):
    file_name = f"match_day_{i}.json"
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for match in data:

        base_text = f"{match.get("matchID")} {match.get("leagueName")} - {match.get("matchDateTime")} - {match.get("group").get("groupName")} - {match.get("team1").get("teamName")} vs {match.get("team2").get("teamName")} - {match.get("location").get("locationStadium") if match.get("location") else match.get("team1").get("shortName")} - "
        for matchResult in match.get("matchResults"):
            tmp_text = base_text + f"{matchResult.get("resultName")} - {matchResult.get("pointsTeam1")} : {matchResult.get("pointsTeam2")}"
            texts.append(tmp_text)
        for goal in match.get("goals"):
            tmp_text = base_text + f"{goal.get("goalID")} - {goal.get("scoreTeam1")} : {goal.get("scoreTeam2")}  - matchMinute={goal.get("matchMinute")} - {goal.get("goalGetterName")} - isPenalty={goal.get("isPenalty")} - isOwnGoal={goal.get("isOwnGoal")} - isOvertime={goal.get("isOvertime")}"
            texts.append(tmp_text)
        if len(match.get("goals")) + len( match.get("matchResults")) == 0:
            texts.append(base_text)



In [ ]:
df = pd.DataFrame({"text": texts})

print(df.head())

                                                text
0  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...
1  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...
2  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...
3  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...
4  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...


In [ ]:
df.to_csv("bundesliga_matches.csv", index=False, encoding="utf-8")

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [3]:

#imports

import pandas as pd
import openai
import ast
from scipy.spatial.distance import cosine

In [ ]:
df = pd.read_csv("bundesliga_matches.csv", encoding="utf-8")

In [ ]:
client = openai.OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = "YOUR API KEY"
)

embeddings = []

for text in df["text"]:
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small" 
    )
    embedding_vector = response.data.get("embedding")
    embeddings.append(embedding_vector)

df["embedding"] = embeddings
df.head()

# Optional: CSV inkl. Embeddings abspeichern
df.to_csv("bundesliga_matches_with_embeddings.csv", index=False)

                                                text  \
0  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...   
1  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...   
2  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...   
3  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...   
4  72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...   

                                           embedding  
0  [Embedding(embedding=[-0.01921803131699562, 0....  
1  [Embedding(embedding=[-0.007300814613699913, 0...  
2  [Embedding(embedding=[-0.03225141018629074, 0....  
3  [Embedding(embedding=[-0.026533454656600952, 0...  
4  [Embedding(embedding=[-0.03140493854880333, 0....  


In [ ]:
# dev tool
df = pd.read_csv("bundesliga_matches_with_embeddings.csv", encoding="utf-8")
df["embedding"] = df["embedding"].apply(lambda x: ast.literal_eval(x[21:-31]))


df.head()

,text,embedding
0,72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...,"[-0.01921803131699562, 0.014085751958191395, 0..."
1,72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...,"[-0.007300814613699913, 0.009188768453896046, ..."
2,72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...,"[-0.03225141018629074, 0.01771959662437439, 0...."
3,72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...,"[-0.026533454656600952, 0.023782579228281975, ..."
4,72214 1. Fußball-Bundesliga 2024/2025 - 2024-0...,"[-0.03140493854880333, 0.029117325320839882, 0..."


In [ ]:
client = openai.OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = "YOUR API KEY"
)

def get_context(question, limit = 50):
    df = pd.read_csv("bundesliga_matches_with_embeddings.csv", encoding="utf-8")
    df["embedding"] = df["embedding"].apply(lambda x: ast.literal_eval(x[21:-31]))
    searching_embeddings = get_embeded_question(question)
    df["distances"] = df['embedding'].apply(lambda embedding: cosine(embedding, searching_embeddings))
    df.sort_values("distances", ascending=True, inplace=True)
    return df['text'].head(limit).tolist()

def get_embeded_question(question):
    response = client.embeddings.create(
        input=question,
        model="text-embedding-3-small" 
    )
    return response.data[0].embedding
    



def answer_without_context(question):
    prompt = [
        {
            'role': 'user',
            'content': question
        }
    ]
    return request_openai(prompt)

def answer_with_context(question):
    prompt =  [
        {
            'role': 'system',
            'content': f"""
            Answer the question based on your own knowledege or the provided context below. If you dont know the answer, say "I don't know". The context provides facts about the season 2024/2025. Facts are annotated with date and seperated by lines. 
            Context: 
                {get_context(question)}
            """
        },
        {
            'role': 'user',
            'content': question
        }
    ]
    return request_openai(prompt)

def request_openai(prompt, client=client, model_name="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        max_tokens=100
    )
    return response.choices[0].message.content




## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [10]:
def compare_models(question):
    print("Answer without context")
    print(answer_without_context(question))

    print("Answer with context")
    print(answer_with_context(question))

### Easy questions

In [26]:
question = "Who did Borrusia Dortmund play against on matchday 25 of the season 24/25?"
compare_models(question)

Answer without context
I'm sorry, but as an AI language model, I do not have access to real-time data such as sports scores or schedules. You can check the official Borussia Dortmund website or a sports news website to find out who they played against on matchday 25 of the season 24/25.
Answer with context
Borussia Dortmund played against FC Augsburg on matchday 25 of the season 24/25.


In [16]:
question = "Which matches were on the 25. matchday of the Bundesliga of the season 24/25?"
compare_models(question)

Answer without context
I'm sorry, I cannot provide the specific matches that were played on the 25th matchday of the Bundesliga season 24/25 as that information is not readily available. Would you like to know about a different Bundesliga season or matchday?
Answer with context
The matches on the 25th matchday of the Bundesliga season 24/25 were: 
1. Bayer Leverkusen vs Werder Bremen
2. FC Bayern München vs VfL Bochum


### Hard questions

In [37]:
question = "how often did bayern munich lose in the bundesliga in the season 24/25? And print the match ids."
compare_models(question)

# wrong answer

Answer without context
To answer this question, I would need access to a database or detailed records of all Bayern Munich matches in the 24/25 Bundesliga season in order to identify all the matches they lost and then print the match IDs. Unfortunately, I do not have access to live data at the moment. I recommend checking official Bundesliga websites or sports databases for this specific information.
Answer with context
Bayern Munich lost one game in the Bundesliga in the 2024/2025 season. The match ID where Bayern Munich lost is as follows:
- Match ID: 72250


In [19]:
question = "who scores the most goals of the bundesliga in the season 24/25?"
compare_models(question)

# wrong answer

Answer without context
It is not possible to predict or determine who will score the most goals in the Bundesliga in the 24/25 season as it has not happened yet. The leading goal scorer can vary from season to season based on the performance of individual players and teams.
Answer with context
In the Bundesliga season 24/25, the player who scored the most goals was Patrik Schick from Bayer Leverkusen.
